<a href="https://colab.research.google.com/github/qin-na/SZPT-Q/blob/main/%E5%9F%BA%E7%A1%80%E7%BB%83%E4%B9%A02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.datasets import mnist
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()
print(train_images.shape)
digit = test_images[0]
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

In [ ]:
import numpy as np
samples = ['The cat jump over the dog', 'The dog ate my homework']

#我们先将每个单词放置到一个哈希表中
token_index = {}
for sample in samples:
    #将一个句子分解成多个单词
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
            
#设置句子的最大长度
max_length = 10
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[: max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.
        print("{0} -> {1}".format(word, results[i, j]))
from keras.preprocessing.text import Tokenizer

def oneHotEncode(samples):
    #只考虑最常使用的前1000个单词
    tokenizer = Tokenizer(num_words = 1000)
    tokenizer.fit_on_texts(samples)
    #把句子分解成单词数组
    sequences = tokenizer.texts_to_sequences(samples)
    return sequences

samples = ['The cat jump over the dog', 'The dog ate my homework']
vecs = oneHotEncode(samples)
print(vecs)


In [9]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt

#加载手写数字图片数据
(x_train, _), (x_test, _) = mnist.load_data()
image_size = x_train.shape[1]


#把图片大小统一转换成28*28,并把像素点值都转换为[0,1]之间
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

input_shape = (image_size, image_size, 1)
batch_size = 32
#对图片做3*3分割
kernel_size = 3
#让编码器将输入图片编码成含有16个元素的向量
latent_dim = 16
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
'''
编码器含有两个卷积层，第一个卷积层将每个3*3切片计算成含有32个元素的向量，第二个卷积层将3*3切片
计算成含有64个元素的向量
'''

layer_filters = [32, 64]
for filters in layer_filters:
  #stride=2表明每次挪到2个像素，如此一来做一次卷积运算后输出大小会减半
  x = Conv2D(filters = filters, kernel_size = kernel_size, activation='relu',
            strides = 2,
            padding = 'same')(x)

shape = K.int_shape(x)
print('shape: ', shape)
print(shape[1])
x = Flatten()(x)
#最后一层全连接网络输出含有16个元素的向量
latent = Dense(latent_dim, name = 'latent_vector')(x)
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

11490434/11490434 [==============================] - 0s 0us/step
shape:  (None, 7, 7, 64)
7
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 latent_vector (Dense)       (None, 16)                50192     
                                                                 
Total params: 69,008
Trainable params: 69,008
Non-trainable params: 0
_____________________________